<a href="https://colab.research.google.com/github/0xVolt/whats-up-doc/blob/main/test/notebooks/model-blending/Llama-3-8B-CodeLlama-7b-Instruct-hf-passthrough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Merge the `Llama-3-8B` and `CodeLlama-7B-instruct-hf` Models using the Passthrough Method

## Download and Install `mergekit`

In [1]:
import os

dirName = "mergekit"
cwd = os.getcwd()

concatDirPath = os.path.join(cwd, dirName)

if not os.path.exists(concatDirPath):
    !git clone https://github.com/cg123/mergekit.git
    !cd mergekit && pip install -q -e .

Cloning into 'mergekit'...
remote: Enumerating objects: 2265, done.
remote: Counting objects: 100% (1354/1354), done.
remote: Compressing objects: 100% (520/520), done.
remote: Total 2265 (delta 1081), reused 947 (delta 833), pack-reused 911
Receiving objects: 100% (2265/2265), 640.50 KiB | 18.84 MiB/s, done.
Resolving deltas: 100% (1584/1584), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 33.2 MB/

In [2]:
import os
import yaml
from transformers import AutoModelWithLMHead, AutoTokenizer, pipeline

In [3]:
%pip install huggingface-cli

*Note: The token you generate and enter into this login needs to be a write token from huggingface in order to create a new repository using the API and to push the merged model's `safetensors` to the repo*

In [14]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write)

## Create the YAML Config File to Merge Models with Passthrough

In [8]:
# Set the resultant model's name
MODEL_NAME = 'whats-up-llamas'

MODEL_1 = "codellama/CodeLlama-7b-Instruct-hf"
MODEL_2 = "meta-llama/Meta-Llama-3-8B-Instruct"

OUTPUT_DIR = "merged_model"

### Write Config Script

In [9]:
yamlConfigPassthrough = f"""
slices:
  - sources:
    - model: {MODEL_1}
      layer_range: [0, 32]
  - sources:
    - model: {MODEL_2}
      layer_range: [24, 32]
merge_method: passthrough
dtype: bfloat16

"""

*Note: If you were to do this locally, instead of putting in the models' card name under `model`, you would specify the path to the model you downloaded from huggingface.*

### Save Config Script

In [10]:
# Save the YAML configuration to a file
yamlFileName = "config.yaml"
with open(yamlFileName, "w") as f:
    f.write(yamlConfigPassthrough)

## Merge Models

In [11]:
cmd = f"mergekit-yaml {yamlFileName} {OUTPUT_DIR} --allow-crimes --copy-tokenizer --out-shard-size 1B --low-cpu-memory --write-model-card --lazy-unpickle"
!{cmd}

Streaming output truncated to the last 5000 lines.

model-00001-of-00002.safetensors:  33% 3.33G/9.98G [00:46<08:42, 12.7MB/s]



model-00002-of-00002.safetensors:  87% 3.04G/3.50G [00:46<00:42, 10.9MB/s]


model-00001-of-00002.safetensors:  34% 3.37G/9.98G [00:46<05:14, 21.0MB/s]



model-00002-of-00002.safetensors:  87% 3.06G/3.50G [00:46<00:29, 15.0MB/s]



model-00002-of-00002.safetensors:  88% 3.08G/3.50G [00:47<00:20, 20.6MB/s]


model-00001-of-00002.safetensors:  34% 3.40G/9.98G [00:47<03:27, 31.7MB/s]


model-00001-of-00002.safetensors:  34% 3.42G/9.98G [00:47<02:43, 40.2MB/s]



model-00002-of-00002.safetensors:  89% 3.10G/3.50G [00:47<00:14, 27.6MB/s]



model-00002-of-00002.safetensors:  89% 3.12G/3.50G [00:47<00:11, 33.3MB/s]


model-00001-of-00002.safetensors:  34% 3.44G/9.98G [00:47<02:26, 44.6MB/s]



model-00002-of-00002.safetensors:  90% 3.15G/3.50G [00:47<00:09, 36.8MB/s]


model-00001-of-00002.safetensors:  35% 3.46G/9.98G [00:47<02:22, 45.6MB/s]



model-00002-of-00

## Save the Model on `huggingface` as a `model` Repo

In [22]:
# Use userdata to access the HF_TOKEN as an env variable
from google.colab import userdata
from huggingface_hub import HfApi

username = "0xVolt"

# Defined in the secrets tab in Google Colab
api = HfApi(token=userdata.get("HF_TOKEN"))

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)

# Push the whole merged-model folder to the hub
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path=OUTPUT_DIR,
)

model-00001-of-00019.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00003-of-00019.safetensors:   0%|          | 0.00/910M [00:00<?, ?B/s]

model-00002-of-00019.safetensors:   0%|          | 0.00/971M [00:00<?, ?B/s]

model-00004-of-00019.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Upload 20 LFS files:   0%|          | 0/20 [00:00<?, ?it/s]

...

model-00017-of-00019.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00018-of-00019.safetensors:   0%|          | 0.00/944M [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/405M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/0xVolt/whats-up-llamas/commit/78f7a842606606fce3af3de0491d416b23bd7592', commit_message='Upload folder using huggingface_hub', commit_description='', oid='78f7a842606606fce3af3de0491d416b23bd7592', pr_url=None, pr_revision=None, pr_num=None)